In [1]:
import vantage6.client as v6client
from time import sleep
import os

In [2]:
v6_host = 'https://' + os.environ['V6_HOST']
v6_user = os.environ['V6_USER']
v6_password = os.environ['V6_PASSWORD']
v6_key = os.environ['V6_KEY']

In [3]:
client = v6client.Client(v6_host, None)

client.authenticate(v6_user, v6_password)
client.setup_encryption(v6_key)

In [4]:
nodes = client.node.list(is_online=True)
nodes

{'data': [{'ip': '10.155.40.146',
   'type': 'node',
   'status': 'online',
   'name': 'legacy-lamb - windy-emporium Node',
   'last_seen': '2022-08-02T09:14:47.022491+00:00',
   'organization': {'id': 2,
    'link': '/api/organization/2',
    'methods': ['GET', 'PATCH']},
   'id': 2,
   'collaboration': {'id': 1,
    'link': '/api/collaboration/1',
    'methods': ['PATCH', 'DELETE', 'GET']}},
  {'ip': '10.155.40.147',
   'type': 'node',
   'status': 'online',
   'name': 'regular-pass - windy-emporium Node',
   'last_seen': '2022-08-02T09:50:39.699454+00:00',
   'organization': {'id': 3,
    'link': '/api/organization/3',
    'methods': ['GET', 'PATCH']},
   'id': 3,
   'collaboration': {'id': 1,
    'link': '/api/collaboration/1',
    'methods': ['PATCH', 'DELETE', 'GET']}},
  {'ip': '10.155.40.145',
   'type': 'node',
   'status': 'online',
   'name': 'noisy-ancestor - windy-emporium Node',
   'last_seen': '2022-08-02T09:50:38.711716+00:00',
   'organization': {'id': 5,
    'link': '

In [6]:
orgs = [n['id'] for n in nodes['data']]
orgs

[2, 3, 5]

In [11]:
verticox_image = 'harbor.carrier-mu.src.surf-hosted.nl/carrier/verticox'



input_params = {'method': 'verticox', 'master': True, 'kwargs':
                {
                    'feature_columns': ['afb', 
#                                        'age', 'av3', 'bmi', 'chf', 'cvd', 
                    #                     'diasbp', 'gender', 'hr','los', 'miord',
                    #                     'mitype', 'sho', 
                                        'sysbp'],
                    'event_times_column': 'event_time',
                    'event_happened_column': 'event_happened',
                     'datanode_ids': orgs[1:],
                    'precision':0.1
                }
               }


task = client.task.create(collaboration=1, organizations=[orgs[0]], name='verticox', 
                          image=verticox_image, description='verticox test',
                   input=input_params)
                   

In [13]:
result_id = task['results'][0]['id']

r = client.result.get(result_id)

while not r['finished_at']:
    sleep(3)
    r = client.result.get(result_id)
    
print(f'Result:\n{r["result"]}')
print(f'Log:\n{r["log"]}')
      

Result:
b''
Log:
info > wrapper for verticox
info > Reading input file /mnt/data/task-000000101/input
info > No data format specified. Assuming input data is pickle format
info > Reading token file '/mnt/data/task-000000101/token'
info > Using '/mnt/data/default.csv' as database
info > Dispatching ...
info > Module 'verticox' imported!
info > Running a master-container
info > Working with collaboration_id <1>
info > Dispatching node-tasks
info > Connecting to datanode at 10.155.40.147:49158
info > Connecting to datanode at 10.155.40.145:49158
[info]  - Starting iteration num 0
[info]  - z_diff: 10.417891704212444
[info]  - sigma_diff: 20.421355810565068
[info]  - Starting iteration num 1
warn > Error encountered while calling verticox: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1659448994.887446248","description":"Failed to pick subchannel","file":"src/core/ext/filt

In [50]:
r = client.result.get(result_id)
r['finished_at']

'2022-07-28T12:12:42.502165+00:00'

In [42]:
client.task.list()['data'][0]

{'complete': True,
 'results': [{'id': 25,
   'link': '/api/result/25',
   'methods': ['GET', 'PATCH']}],
 'children': None,
 'database': 'default',
 'initiator': 6,
 'parent': None,
 'name': 'verticox',
 'description': 'verticox test',
 'image': 'harbor.carrier-mu.src.surf-hosted.nl/carrier/verticox:0.1.0',
 'run_id': 13,
 'id': 25,
 'collaboration': {'id': 1,
  'link': '/api/collaboration/1',
  'methods': ['PATCH', 'DELETE', 'GET']}}

In [8]:
for t in client.task.list()['data']:
    id = t['id']
    client.task.delete(id)